In [2]:
from lyricsgenius import Genius
import lyricsgenius as lg
import requests
token = 'vEuhOt5S8b7JMcQuXZhlSNouoVIPQruY6a0bsie-lifgyPxsx_dPVzDSZools0pG'


In [3]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time
import random
import os


GENIUS_API_TOKEN = 'vEuhOt5S8b7JMcQuXZhlSNouoVIPQruY6a0bsie-lifgyPxsx_dPVzDSZools0pG'
genius = lg.Genius(GENIUS_API_TOKEN)

def get_artist_id(artist_name):
    """Récupère l'ID d'un artiste en utilisant l'API Genius."""
    search_url = f"https://api.genius.com/search?q={artist_name}"
    headers = {'Authorization': 'Bearer ' + GENIUS_API_TOKEN}
    
    try:
        response = requests.get(search_url, headers=headers, timeout=180)
        response.raise_for_status()  # Vérifie les erreurs HTTP
        data = response.json()
        
        # Vérifie si la réponse contient la clé 'response'
        if 'response' in data and data['response']['hits']:
            return data['response']['hits'][0]['result']['primary_artist']['id']
        else:
            print(f"Aucune donnée trouvée pour l'artiste: {artist_name}")
            return None
    except Exception as e:
        print(f"Erreur lors de la récupération de l'ID pour {artist_name}: {e}")
        return None

def get_songs(artist_id, max_songs=500):
    """Récupère jusqu'à un nombre donné de chansons pour un artiste, triées par popularité."""
    songs = []
    page = 1
    
    while len(songs) < max_songs:
        url = f"https://api.genius.com/artists/{artist_id}/songs?page={page}"
        headers = {'Authorization': 'Bearer ' + GENIUS_API_TOKEN}
        response = requests.get(url, headers=headers).json()
        songs_data = response['response']['songs']
        
        if not songs_data:
            break  # Sortir si aucune chanson n'est trouvée

        # Récupérer la popularité et ajouter les chansons à la liste
        for song in songs_data:
            if len(songs) < max_songs:  # Assurez-vous de ne pas dépasser max_songs
                songs.append({
                    'title': song['title'],
                    'popularity': song.get('stats', {}).get('pageviews', 0)  # Exemple de mesure de popularité
                })
            else:
                break  # Sortir de la boucle for si le maximum est atteint

        page += 1  # Passer à la page suivante
        time.sleep(random.uniform(1,3))
        
    # Trier manuellement par popularité
    sorted_songs = sorted(songs, key=lambda x: x['popularity'], reverse=True)
    return [song['title'] for song in sorted_songs[:max_songs]]  # Limiter le retour à max_songs

def download_lyrics(song_title, artist_name):
        """Télécharge les paroles d'une chanson donnée."""

        song = genius.search_song(song_title, artist_name)
        if song : 
            try :
                paroles = song.lyrics
            except Exception as e :
                print(f"Erreur lors de la récupération des paroles : {e}")
                return None
        
            return paroles, artist_name, song_title
        return None

import os

import os
import pandas as pd

def all_artist_songs(artist_to_scrape):
    """Télécharge les paroles de toutes les chansons pour chaque artiste et les stocke dans un fichier CSV par artiste."""
    for artist in artist_to_scrape:
        num = 1
        artist_id = get_artist_id(artist)
        if artist_id is not None:
            songs = get_songs(artist_id)
            if songs:
                # Créer un répertoire pour l'artiste si il n'existe pas
                os.makedirs("Artiste", exist_ok=True)
                artist_lyrics = []  # Liste pour stocker les données des chansons

                for song in songs:
                    lyrics_info = download_lyrics(song, artist)  # Modifiez cette ligne pour obtenir les infos des paroles
                    if lyrics_info:
                        lyrics, artist, song = lyrics_info
                        artist_lyrics.append({
                            'Artist': artist,
                            'Title': song,
                            'Lyrics': lyrics
                        })  # Ajouter les informations de la chanson
                        print(f"Téléchargé : {song} de {artist}")
                        print(f"Chanson numéro : {num}")
                        num +=1 
                        sleep_time = random.uniform(0.5,1.5)
                        print(f"Sleeping for {sleep_time:.2f} seconds before the next download...")
                        time.sleep(sleep_time)
                    else:
                        print(f"Paroles non trouvées pour {song} de {artist}")

                # Créer un DataFrame à partir de la liste des paroles
                df = pd.DataFrame(artist_lyrics)

                # Enregistrer le DataFrame dans un fichier CSV
                df.to_csv(f"Artiste/{artist}_songs.csv", index=False, encoding='utf-8')
                print(f"Fichier créé : {artist}/{artist}_songs.csv")
            else:
                print(f"Aucune chanson trouvée pour {artist}")
        else:
            print(f"ID d'artiste non trouvé pour {artist}")

# Liste d'artistes à scraper
#   # Remplacez par votre liste d'artistes



In [6]:
artist_to_scrape = ["Alpha Wann", "Menelik", "Mister You", "Nekfeu", ]

In [7]:
all_artist_songs(artist_to_scrape)

Searching for "Monsieur Sable" by Alpha Wann...
Done.
Téléchargé : Monsieur Sable de Alpha Wann
Chanson numéro : 1
Sleeping for 0.96 seconds before the next download...
Searching for "Du sexe opposé" by Alpha Wann...
Done.
Téléchargé : Du sexe opposé de Alpha Wann
Chanson numéro : 2
Sleeping for 0.55 seconds before the next download...
Searching for "Ne le dis pas à ma mère" by Alpha Wann...
Done.
Téléchargé : Ne le dis pas à ma mère de Alpha Wann
Chanson numéro : 3
Sleeping for 1.42 seconds before the next download...
Searching for "Flingue & Feu" by Alpha Wann...
Done.
Téléchargé : Flingue & Feu de Alpha Wann
Chanson numéro : 4
Sleeping for 1.49 seconds before the next download...
Searching for "Aux portes du succès" by Alpha Wann...
Done.
Téléchargé : Aux portes du succès de Alpha Wann
Chanson numéro : 5
Sleeping for 1.11 seconds before the next download...
Searching for "En soum soum" by Alpha Wann...
Done.
Téléchargé : En soum soum de Alpha Wann
Chanson numéro : 6
Sleeping for 1.0

Timeout: Request timed out:
HTTPSConnectionPool(host='genius.com', port=443): Read timed out. (read timeout=5)

In [6]:
import os
import shutil

def copy_csv_to_all_songs(source_directory, target_directory="all_songs"):
    # Crée le dossier de destination s'il n'existe pas
    os.makedirs(target_directory, exist_ok=True)

    # Itérer à travers tous les sous-dossiers dans le dossier source
    for root, dirs, files in os.walk(source_directory):
        for file in files:
            # Vérifier si le fichier est un CSV
            if 'songs' in file:
                # Construire le chemin complet du fichier source
                source_file = os.path.join(root, file)
                
                # Copier le fichier dans le dossier cible
                shutil.copy(source_file, target_directory)
                print(f"Copié : {source_file} vers {target_directory}")

# Exemple d'utilisation
source_dir = r"C:\Users\carlf\Documents\GitHub\Lyrics-Generator-"  # Remplacez par le chemin de votre dossier contenant les sous-dossiers
copy_csv_to_all_songs(source_dir)


SameFileError: 'C:\\Users\\carlf\\Documents\\GitHub\\Lyrics-Generator-\\all_songs\\ATEYABA_songs.csv' and 'all_songs\\ATEYABA_songs.csv' are the same file

In [ ]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# Configuration de l'API Spotipy
client_id = 'YOUR_CLIENT_ID'
client_secret = 'YOUR_CLIENT_SECRET'
credentials = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=credentials)

# Fonction pour récupérer le genre d'une chanson
def get_song_genre(artist, title):
    # Recherche de l'artiste
    results = sp.search(q=f'artist:{artist} track:{title}', type='track', limit=1)
    if results['tracks']['items']:
        track = results['tracks']['items'][0]
        # Récupérer l'ID de l'album pour obtenir le genre
        album_id = track['album']['id']
        album = sp.album(album_id)
        # Les genres sont dans la clé 'genres' de l'album
        return album['genres']  # Cela renvoie une liste de genres
    return None

# Itération dans le DataFrame et ajout des genres
df['Genre'] = df.apply(lambda row: get_song_genre(row['Artist'], row['Title']), axis=1)

# Affichage du DataFrame avec les genres
print(df)


In [2]:
from lyricsgenius import Genius
import lyricsgenius as lg
import requests
import requests
import pandas as pd 
import numpy as np
import random 
from bs4 import BeautifulSoup

from tqdm import tqdm 
import os


token = 'vEuhOt5S8b7JMcQuXZhlSNouoVIPQruY6a0bsie-lifgyPxsx_dPVzDSZools0pG'
GENIUS_API_TOKEN = 'vEuhOt5S8b7JMcQuXZhlSNouoVIPQruY6a0bsie-lifgyPxsx_dPVzDSZools0pG'
genius = lg.Genius(GENIUS_API_TOKEN)
artist_to_scrape = ["BOOBA"]


class ArtistLyricsScraper:
    def __init__(self, GENIUS_API_TOKEN, artist_to_scrape) :
        self.GENIUS_API_TOKEN = GENIUS_API_TOKEN
        self.genius = lg.Genius(GENIUS_API_TOKEN)
        self.artist_to_scrape = artist_to_scrape
        
    def get_artist_id(self, artist_name):
        """Récupère l'ID d'un artiste en utilisant l'API Genius."""
        search_url = f"https://api.genius.com/search?q={artist_name}"
        headers = {'Authorization': 'Bearer ' + self.GENIUS_API_TOKEN}
        
        try:
            response = requests.get(search_url, headers=headers)
            response.raise_for_status()  # Vérifie les erreurs HTTP
            data = response.json()
            
            # Vérifie si la réponse contient la clé 'response'
            if 'response' in data and data['response']['hits']:
                return data['response']['hits'][0]['result']['primary_artist']['id']
            
            print(f"Aucune donnée trouvée pour l'artiste: {artist_name}")
            return None
        except Exception as e:
            print(f"Erreur lors de la récupération de l'ID pour {artist_name}: {e}")
            return None

    def get_songs(self, artist_id, max_songs=100):
        """Récupère jusqu'à un nombre donné de chansons pour un artiste, triées par popularité."""
        songs = []
        page = 1
        while len(songs) < max_songs:
            url = f"https://api.genius.com/artists/{artist_id}/songs?page={page}"
            headers = {'Authorization': 'Bearer ' + self.GENIUS_API_TOKEN}
            response = requests.get(url, headers=headers).json()
            songs_data = response['response']['songs']
            if not songs_data:
                break

            # Récupérer la popularité et trier manuellement (si disponible)
            for song in songs_data:
                if len(songs) >= max_songs:  # Stopper si on atteint la limite
                    break
                songs.append({
                    'title': song['title'],
                    'popularity': song.get('stats', {}).get('pageviews', 0)  # Exemple de mesure de popularité
                })

            page += 1

        # Trier manuellement par popularité
        sorted_songs = sorted(songs, key=lambda x: x['popularity'], reverse=True)
        return [song['title'] for song in sorted_songs]

    def download_lyrics(self, song_title, artist_name):
        """Télécharge les paroles d'une chanson donnée."""

        song = self.genius.search_song(song_title, artist_name)
        if song : 
            try :
                paroles = song.lyrics
            except Exception as e :
                return None
        
            return paroles
        return None

    def all_artist_songs(self, artist_to_scrape):
        """Télécharge les paroles de toutes les chansons pour 
        chaque artiste et les stocke dans un fichier par artiste."""
    for artist in artist_to_scrape:
        artist_id = self.get_artist_id(artist)
        if artist_id is not None:
            songs = get_songs(artist_id)
            if songs:
                # Créer un répertoire pour l'artiste si il n'existe pas
                os.makedirs(artist, exist_ok=True)
                artist_lyrics = []  # Liste pour stocker les paroles de toutes les chansons

                for song in songs:
                    lyrics = self.download_lyrics(song, artist)
                    if lyrics:
                        artist_lyrics.append(f"{song}\n{lyrics}\n")  # Ajouter les paroles au contenu de l'artiste
                        print(f"Téléchargé : {song} de {artist}")
                    else:
                        print(f"Paroles non trouvées pour {song} de {artist}")

                # Créer un fichier texte pour l'artiste contenant toutes les chansons
                with open(f"{artist}/{artist}_songs.txt", 'w', encoding='utf-8') as file:
                    file.write("\n".join(artist_lyrics))
                    print(f"Fichier créé : {artist}/{artist}_songs.txt")
            else:
                print(f"Aucune chanson trouvée pour {artist}")
        else:
            print(f"ID d'artiste non trouvé pour {artist}")

NameError: name 'self' is not defined

In [21]:
import pandas as pd 


df = pd.read_csv(r"C:\Users\carlf\Documents\GitHub\Lyrics-Generator-\Artiste\DAMSO_songs.csv")

In [24]:
df.loc[105:125]

,Artist,Title,Lyrics
105,DAMSO,Ο. OG,"39 ContributorsΟ. OG Lyrics[Paroles de ""Ο. OG""..."
106,DAMSO,Coeur De Pirate,35 ContributorsCoeur De Pirate Lyrics[Paroles ...
107,DAMSO,LA RUE,"31 ContributorsLA RUE Lyrics[Paroles de ""LA RU..."
108,DAMSO,Iscariote,49 ContributorsTranslationsEnglishIscariote Ly...
109,DAMSO,COEUR DE ICE,32 ContributorsCOEUR DE ICE Lyrics[Paroles de ...
110,DAMSO,Dégaine,13 ContributorsTranslationsEnglishDégaine Lyri...
111,DAMSO,Σ. Ouzbek*,35 ContributorsΣ. Ouzbek* Lyrics[Paroles issue...
112,DAMSO,Nocif,"26 ContributorsNocif Lyrics[Paroles de ""Nocif""..."
113,DAMSO,Mucho Dinero,48 ContributorsMucho Dinero Lyrics[Paroles de ...
114,DAMSO,YOUVOI,40 ContributorsTranslationsEnglishYOUVOI Lyric...
